# Preprocess Dataset/Generate Dataset

In [134]:
import os
import torch
import pathlib
import shutil
import numpy as np
import PIL
from fastai.vision import *
from torchvision import transforms

In [135]:
DIV2K_path = pathlib.Path('.').parent.absolute()
train_HR = DIV2K_path/'DIV2K_train_HR'
valid_HR = DIV2K_path/'DIV2K_valid_HR'
assert DIV2K_path.exists() and train_HR.exists() and valid_HR.exists()

In [136]:
high_res = 128
low_res_factor = 4
num_extra = 2
same_size = False
low_res = high_res // low_res_factor
output_path = DIV2K_path/'same_size'/f'{high_res}x{low_res_factor}'
print(f'high resolution: {high_res}')
print(f'low resolution: {low_res}')
print(f'output path: {output_path}')

high resolution: 128
low resolution: 32
output path: /home/hacker/Documents/Super-Resolution/datasets/DIV2K/same_size/128x4


In [137]:
# clean and create output_path
if output_path.exists():
    shutil.rmtree(output_path)
output_path.mkdir(parents=True, exist_ok=True)

In [138]:
# clean and create subfolders
hr_train = output_path/'hr_train'
hr_valid = output_path/'hr_valid'
lr_train = output_path/'lr_train'
lr_valid = output_path/'lr_valid'
for dir_path in [hr_train, hr_valid, lr_train, lr_valid]:
    if os.path.exists(dir_path):
        shutil.rmtree(dir_path)
    dir_path.mkdir(parents=True, exist_ok=True)

## Get Image Info

In [139]:
hr_valid_image_list = ImageList.from_folder(valid_HR)
hr_train_image_list = ImageList.from_folder(train_HR)
hr_train_image_name_list = [img_path.relative_to(train_HR) for img_path in hr_train_image_list.items]
shapes = [PIL.Image.open(img_path).size for img_path in hr_train_image_list.items]
print(f"min dimension of all training images={torch.min(torch.tensor(shapes))}")
hr_valid_image_name_list = [img_path.relative_to(valid_HR) for img_path in hr_valid_image_list.items]
shapes = [PIL.Image.open(img_path).size for img_path in hr_valid_image_list.items]
print(f"min dimension of all validation images={torch.min(torch.tensor(shapes))}")

min dimension of all training images=648
min dimension of all validation images=816


## Image Agumentation

In [140]:
def get_proper_size_center_crop(image, size: int):
    return transforms.Compose([
        transforms.CenterCrop(min(image.size)),
        transforms.Resize(size, interpolation=PIL.Image.BICUBIC)
    ])(image)

In [141]:
class ImageAugmentor(object):
    def __init__(self, dest_dir, size: int, num_extra:int=2):
        assert num_extra <= 5
        self.num_extra = num_extra
        self.dest_dir = dest_dir
        self.size = size
        self.base_image, self.image_name, self.filename_no_ext, self.ext = None, None, None, None
        self.generated_images, self.generated_image_names = [], []


    def generate(self):
        random_choices = np.random.choice(a=[1, 2, 3, 4, 5], replace=False, size=self.num_extra)
        for choice in random_choices:
            if choice == 1:
                self.generated_images.append(transforms.functional.hflip(self.base_image))
                self.generated_image_names.append(self.filename_no_ext + '-hf' + self.ext)
            elif choice == 2:
                self.generated_images.append(transforms.functional.vflip(self.base_image))
                self.generated_image_names.append(self.filename_no_ext + '-vf' + self.ext)
            elif choice == 3:
                self.generated_images.append(transforms.functional.rotate(self.base_image, 90))
                self.generated_image_names.append(self.filename_no_ext + '-r90' + self.ext)
            elif choice == 4:
                self.generated_images.append(transforms.functional.rotate(self.base_image, 180))
                self.generated_image_names.append(self.filename_no_ext + '-r180' + self.ext)
            elif choice == 5:
                self.generated_images.append(transforms.functional.rotate(self.base_image, 270))
                self.generated_image_names.append(self.filename_no_ext + '-r270' + self.ext)

    def save(self):
        for i in range(len(self.generated_image_names)):
            output_path = self.dest_dir/self.generated_image_names[i]
            self.generated_images[i].save(output_path)

    def __call__(self, image_path, i):
        self.image_name = os.path.basename(image_path)
        self.filename_no_ext, self.ext = os.path.splitext(self.image_name)
        self.base_image = get_proper_size_center_crop(PIL.Image.open(image_path), self.size)
        self.generated_images = [self.base_image]
        self.generated_image_names = [self.image_name]
        self.generate()
        self.save()


In [142]:
# transform and save images
parallel(ImageAugmentor(hr_valid, high_res, num_extra=0), hr_valid_image_list.items)
parallel(ImageAugmentor(hr_train, high_res, num_extra=num_extra), hr_train_image_list.items)

In [143]:
new_hr_valid_image_list = ImageList.from_folder(hr_valid)
new_hr_train_image_list = ImageList.from_folder(hr_train)
new_hr_valid_image_names_list = [img_path.relative_to(hr_valid) for img_path in new_hr_valid_image_list.items]
shapes = [PIL.Image.open(img_path).size for img_path in new_hr_valid_image_list.items]
print(f"min dimension of all generated HR training images={torch.min(torch.tensor(shapes))}")
new_hr_train_image_names_list = [img_path.relative_to(hr_train) for img_path in new_hr_train_image_list.items]
shapes = [PIL.Image.open(img_path).size for img_path in new_hr_train_image_list.items]
print(f"min dimension of all generated HR validation images={torch.min(torch.tensor(shapes))}")

min dimension of all generated HR training images=128
min dimension of all generated HR validation images=128


## Generate Low Resolution Data

In [144]:
class Downscaler(object):
    def __init__(self, src_path, dest_path, low_res_size, same_size=False):
        self.src_path = src_path
        self.dest_path = dest_path
        self.low_res_size = low_res_size
        self.same_size = same_size

    def __call__(self, image_name, i):
        src_image_path = self.src_path/image_name
        target_image_path = self.dest_path/image_name
        src_img = PIL.Image.open(src_image_path)
        downscaled_img = src_img.resize((self.low_res_size, self.low_res_size), resample=PIL.Image.BICUBIC).convert('RGB')
        if self.same_size:
            downscaled_img = downscaled_img.resize((src_img.size[0], src_img.size[0]), resample=PIL.Image.BICUBIC).convert('RGB')
      
        downscaled_img.save(target_image_path)

In [145]:
# transform and save images
parallel(Downscaler(hr_valid, lr_valid, low_res, same_size), new_hr_valid_image_names_list)
parallel(Downscaler(hr_train, lr_train, low_res, same_size), new_hr_train_image_names_list)